In [1]:
from ib_insync import *
import random

In [2]:
util.startLoop()

In [3]:
ib = IB()
ib.connect('127.0.0.1', 4002, readonly=True, clientId=random.randint(1, 100000))

<IB connected to 127.0.0.1:4002 clientId=69473>

In [4]:
underlying = Future('ES', '202003', exchange='GLOBEX', currency='USD')
ib.qualifyContracts(underlying)

[Future(conId=346577697, symbol='ES', lastTradeDateOrContractMonth='20200320', multiplier='50', exchange='GLOBEX', currency='USD', localSymbol='ESH0', tradingClass='ES')]

In [5]:
option_chains = ib.reqSecDefOptParams(
    underlyingSymbol=underlying.symbol,
    futFopExchange=underlying.exchange if underlying.secType == "FUT" else "",
    underlyingSecType=underlying.secType,
    underlyingConId=underlying.conId,
)

util.df(option_chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,GLOBEX,346577697,ES,50,[20200320],"[1200.0, 1250.0, 1300.0, 1350.0, 1400.0, 1450...."
1,GLOBEX,346577697,EW2,50,[20200110],"[1600.0, 1700.0, 1800.0, 1900.0, 1950.0, 2000...."
2,GLOBEX,346577697,EW1,50,[20200103],"[1600.0, 1700.0, 1800.0, 1900.0, 1950.0, 2000...."
3,GLOBEX,346577697,E2C,50,[20200108],"[1600.0, 1700.0, 1800.0, 1900.0, 1950.0, 2000...."
4,GLOBEX,346577697,E1A,50,[20200106],"[1600.0, 1700.0, 1800.0, 1900.0, 1950.0, 2000...."
5,GLOBEX,346577697,E3C,50,[20200115],"[1600.0, 1700.0, 1800.0, 1900.0, 1950.0, 2000...."
6,GLOBEX,346577697,E4A,50,[20191223],"[1600.0, 1700.0, 1800.0, 1900.0, 1950.0, 2000...."
7,GLOBEX,346577697,EW4,50,"[20191227, 20200124]","[100.0, 1100.0, 1600.0, 1700.0, 1800.0, 1900.0..."
8,GLOBEX,346577697,E5A,50,[20191230],"[1600.0, 1700.0, 1800.0, 1900.0, 1950.0, 2000...."
9,GLOBEX,346577697,EW,50,"[20191231, 20200131, 20200228]","[1200.0, 1250.0, 1300.0, 1350.0, 1400.0, 1450...."


In [6]:
limited_option_chains = [c for c in option_chains if c.tradingClass == 'ES']
limited_option_chains

[OptionChain(exchange='GLOBEX', underlyingConId='346577697', tradingClass='ES', multiplier='50', expirations=['20200320'], strikes=[1200.0, 1250.0, 1300.0, 1350.0, 1400.0, 1450.0, 1500.0, 1550.0, 1600.0, 1650.0, 1700.0, 1750.0, 1800.0, 1850.0, 1900.0, 1950.0, 2000.0, 2050.0, 2100.0, 2150.0, 2160.0, 2170.0, 2180.0, 2190.0, 2200.0, 2210.0, 2220.0, 2230.0, 2240.0, 2250.0, 2260.0, 2270.0, 2280.0, 2290.0, 2300.0, 2310.0, 2320.0, 2330.0, 2340.0, 2350.0, 2360.0, 2370.0, 2380.0, 2390.0, 2400.0, 2410.0, 2420.0, 2430.0, 2440.0, 2450.0, 2460.0, 2470.0, 2480.0, 2490.0, 2500.0, 2510.0, 2520.0, 2530.0, 2540.0, 2550.0, 2560.0, 2570.0, 2580.0, 2590.0, 2600.0, 2610.0, 2620.0, 2630.0, 2640.0, 2650.0, 2660.0, 2670.0, 2680.0, 2690.0, 2700.0, 2710.0, 2720.0, 2730.0, 2740.0, 2750.0, 2760.0, 2770.0, 2780.0, 2790.0, 2800.0, 2810.0, 2820.0, 2830.0, 2840.0, 2850.0, 2860.0, 2870.0, 2880.0, 2890.0, 2900.0, 2910.0, 2920.0, 2925.0, 2930.0, 2940.0, 2950.0, 2960.0, 2970.0, 2975.0, 2980.0, 2990.0, 3000.0, 3010.0, 3020

In [7]:
option_contracts = [
    (FuturesOption if underlying.secType == "FUT" else Option)(
        symbol=underlying.symbol,
        lastTradeDateOrContractMonth=expiration,
        strike=strike,
        right=right,
        exchange=chain.exchange,
        multiplier=chain.multiplier,
        tradingClass=chain.tradingClass,
    )
    for chain in limited_option_chains
    if chain.exchange == underlying.exchange
    for expiration in chain.expirations
    for strike in chain.strikes
    for right in ["P", "C"]
]

len(option_contracts)

342

In [8]:
qualified_contracts = ib.qualifyContracts(*option_contracts)
len(qualified_contracts)

342

In [9]:
def onBarUpdate(bars, hasNewBar):
    print(bars[-1])

In [13]:
all_bars = []
for contract in qualified_contracts:
    bars = ib.reqRealTimeBars(contract, 5, 'TRADES', False)
    bars.updateEvent += onBarUpdate
    
    ib.sleep(0)
    all_bars.append(bars)

In [14]:
ib.sleep(30)

True

In [15]:
for bars in all_bars:
    ib.cancelRealTimeBars(bars)